In [20]:
import csv
from math import exp
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns

# Set seed
random.seed(1)

# Logistic Fucction
def logistic(x):
  s = 1/(1 + (exp(-x)))# logistic function over x
  return s

# Calculate Dot Product
def dot(x, y):
  s = sum(a*b for a, b in zip(x,y)) # dot product over x and y
  return s

# Calculate prediction based on model
def predict(model, point):
  p = logistic(dot(model, point['features']))# prediction value returned from logistic function
  return p

#Load Data
def load_file(filename):
  return pd.read_excel(filename)

#Clean Data
def clean_data(dataframe):
  dataframe = dataframe.drop(['CustomerID', 'Country', 'State', 'City', 'Latitude', 'Longitude', 'Count', 'Phone Service', 'Internet Service', 'Contract'], axis=1)
  categories_columns = ['Churn Score', 'CLTV', 'Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Paperless Billing', 'Payment Method', 'Streaming TV', 'Streaming Movies', 'Tech Support', 'Device Protection', 'Online Security', 'Online Backup', 'Multiple Lines', 'Tenure Months', 'Monthly Charges', 'Total Charges', 'Zip Code', 'Lat Long']
  dataframe = pd.get_dummies(dataframe, columns=categories_columns, drop_first = True)
  label_encoder = LabelEncoder()
  dataframe['Churn Label'] = label_encoder.fit_transform(dataframe['Churn Label'])
  X = dataframe.drop(['Churn Label', 'Churn Value', 'Churn Reason'], axis=1)
  y = dataframe['Churn Label']
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  return X_train, X_test, y_train, y_test

#Calculate Accuracy
def evaluate_model(model, y_test, y_pred, y_pred_proba):
  print("Accuracy Score: ", accuracy_score(y_test, y_pred))
  print("Confusion Matrix: ", confusion_matrix(y_test, y_pred))
  print("Classification Report: ", classification_report(y_test, y_pred))
  cm = confusion_matrix(y_test, y_pred)
  plt.figure(figsize=(8, 6))
  plt.title('Confusion Matrix')
  plt.xlabel('Predicted')
  plt.ylabel('Actual')
  sns.heatmap(cm, annot=True, fmt='d', cmap = "crest", xticklabels = ['Not Churn', 'Churn'])
  plt.show()

  fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
  roc_auc = auc(fpr, tpr)
  plt.figure(figsize=(8, 6))
  plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
  plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title(f'ROC Curve for {model.__class__.__name__}')
  plt.legend(loc="lower right")
  plt.show()

#Train and Test Model
def test_model(X_test, y_test, X_train, y_train):
  models = [#LogisticRegression(max_iter = 1000, random_state = 42)
           #DecisionTreeClassifier(random_state = 42)
           #RandomForestClassifier(n_estimators = 100, random_state = 42)
           XGBClassifier(use_label_encoder = False, eval_metric = 'logloss', ramdom_state = 42)]
  for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    evaluate_model(model, y_test, y_pred, y_pred_proba)

    return model


In [ ]:
# prompt: load excel document into dataframe, clean it, and train and test the model. Print metrics

filename = '/content/Telco_customer_churn.xlsx'  # Replace with your file path

# Load the data
dataframe = load_file(filename)

# Clean the data
X_train, X_test, y_train, y_test = clean_data(dataframe)

# Train and test the models, and print the metrics
trained_model = test_model(X_test, y_test, X_train, y_train)